In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import csv
import math
import pickle
import random

# **Read Inputs**

In [21]:
# Read user data
words = []
#list of all possible words that will be used for unigram decoder
all_word = []
with open('Data/data.txt', 'r') as input:
    """
    TODO: Parse user input data to a list of touch point collections/lists 
    """
    touch = ["s", "x", "y"]
    
    text = input.readline()
    w = '' 
    while text:
        if(text[0]!="="):
            z=text.split()
            words.append({touch[0]:z[0], touch[1]:float(z[1]), touch[2]:float(z[2])})
            w += z[0]
        else:
            all_word.append(w)
            w = ''
            
        text=input.readline()
        
words

[{'s': 'i', 'x': 25.258852, 'y': 7.752883},
 {'s': 'f', 'x': 13.50101, 'y': 11.854456},
 {'s': 'a', 'x': 4.9218874, 'y': 13.221646},
 {'s': 't', 'x': 16.132853, 'y': 7.0692873},
 {'s': 'f', 'x': 13.671909, 'y': 12.948208},
 {'s': 'i', 'x': 25.942448, 'y': 7.70731},
 {'s': 'r', 'x': 12.03128, 'y': 7.160434},
 {'s': 's', 'x': 7.48537, 'y': 13.540659},
 {'s': 't', 'x': 15.346718, 'y': 7.3427258},
 {'s': 'y', 'x': 20.91802, 'y': 6.65913},
 {'s': 'o', 'x': 29.633863, 'y': 8.071894},
 {'s': 'u', 'x': 23.754942, 'y': 8.61877},
 {'s': 'f', 'x': 15.244179, 'y': 12.67477},
 {'s': 'a', 'x': 4.648449, 'y': 13.084928},
 {'s': 'i', 'x': 25.703188, 'y': 7.4794445},
 {'s': 'l', 'x': 31.103592, 'y': 13.221646},
 {'s': 'w', 'x': 5.6396623, 'y': 8.482051},
 {'s': 'e', 'x': 8.784202, 'y': 7.6617365},
 {'s': 'r', 'x': 12.03128, 'y': 7.6161637},
 {'s': 'u', 'x': 24.711975, 'y': 7.7984557},
 {'s': 'n', 'x': 24.575256, 'y': 18.189108},
 {'s': 't', 'x': 15.551796, 'y': 8.390905},
 {'s': 'h', 'x': 21.054739, 'y

In [22]:
# Read dictionary
with open('Data/unigram.dict', 'rb') as unigramModelFile:
    unigramModel = pickle.load(unigramModelFile)
unigramModelFile.close()

# Read keyboard data
keyboard_raw = pd.read_csv("Data/keyboard.csv")
keyboard = keyboard_raw[['key', 'x_mm', 'y_mm']]
keyboard


,key,x_mm,y_mm
0,a,4.025010,9.625024
1,b,18.900047,13.650034
2,c,12.950032,13.650034
3,d,9.975025,9.625024
4,e,8.487521,5.600014
5,f,12.950032,9.625024
6,g,15.925039,9.625024
7,h,18.900047,9.625024
8,i,23.362558,5.600014
9,j,21.875053,9.625024


# **Unigram Language Model Decoder**

In [23]:
# Keyboard size and dual Gaussian model parameters
key_width = 3
key_height = 4
a = 2.403
b = 0.017
c = 2.295
d = 0.016

def get_likelihood(p, mu, sigma):
    """
    TODO: Calculate the likelihood that a touch point p is from the 2D Gaussian distribution N(mu, sigma)
    """
    lik = stats.multivariate_normal(mu, sigma).pdf(p)
    return lik

def is_letter(p, letter):
    """
    TODO: Determine if touch point p is located inside the boundary of the key: letter
    """
    keyx = keyboard.iloc[letter]["x_mm"]
    keyy = keyboard.iloc[letter]["y_mm"]
    bufferx = keyx - 2.9750073
    buffery = keyy - 4.02501
    if (p[0] >= keyx and p[0]<=bufferx and p[1] >= keyy and p[1] <= buffery):
        return True
    else:
        return False 

def get_literal_string(touchpoints):
    """ 
    TODO: Compute the literal string using is_letter(p, letter) method for a collection of touch points that represents a word. 
          If a touch point does not fall inside any key boundary, use '?' to represent the corresponding character.
    """
    literal_string = ""
    for tp in touchpoints:
        letter = tp['s']
        p = [tp['x'], tp['y']] 
        if (is_letter(p, letter)):
            literal_string += letter
        else:
            literal_string += "?"
    
    return literal_string

In [24]:
def unigram_lm_decoder(touchpoints):
    """
    A language decoder that uses the dual Gaussian touch point spatial disrtibution model and a unigram language model.
    Input: a list/collection of touch points that represents a certain word
    Output: the decoded word for the input
    
    TODO: Step a --- Get all possible words and their corresponding probabilities from the dictionary. 
          Use the length of the correct word to filter possible words
          You may also use the first and/or the last touchpoint to further narrow down possible words
    """
    
    possible_words = touchpoints
    
    # Calculate p(w|s_1, s_2, ... s_n) ~ p(s_1, s_2, ..., s_n|w)*p(w) = \Pi(p(s_i|c_i))p(w) for each possible word
    p_w_s = []                  # Holds p(w|s_1, s_2, ..., s_n) for all possible words
    for item in possible_words:
        word = item
        p_w = unigramModel["words"]
        p_s_w = 1                 

        for j, letter in enumerate(list(word)):
            
            mu = mean(j)
            sigma = [[a + b * key_width * key_width, 0], [0, c + d * key_height * key_height]]
            p_s_c = get_likelihood(j, mu, sigma)
            
            p_s_w *= p_s_c
            
        temp = p_s_w * p_w    
        p_w_s.append(temp)
    
    
    decoded_word = ""
    decoded_word = max(p_w_s)
    
    return decoded_word

In [25]:
decoded_success_count = 0
literal_success_count = 0
decoded_words = []
literal_strings = []
correct_words = []
for touchpoints in words:
    """
    TODO: Use above methods to compute the correct word, decoded word, and the literal string for each touch point collection
          Append results to the corresponding list
          Update the decoded words/literal strings success count. 
              --- If the decoded word/literal string is the same as correct word, increase 1 to decoded words/literal strings success count
    """
    
# TODO: calculate the success rate for both the decoded words and the literal strings using the docoded word/literal string success count

# TODO: Write to results.txt
with open("results.txt", 'w') as output:
    # The first line: success_rate(decoded_words), success_rate(literal_strings)
    for i in range(len(correct_words)):
        # Each line after: correct_word, decoded_word, literal_string

output.close()

IndentationError: expected an indented block (<ipython-input-25-570736e92bc7>, line 22)